This notebook demonstrates the inference for text summarization using mpt-7b-instruct model.

Getting inference mainly based on defining three steps: model, tokenizer, and pipeline.

In [ ]:
!pip install transformers
!pip install langchain

In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U einops
# !pip install -q -U safetensors
#these dependencies will be for quantization

In [ ]:
import transformers
model = transformers.AutoModelForCausalLM.from_pretrained(
  'mosaicml/mpt-7b-instruct',
  trust_remote_code=True
)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/925e0d80e50e77aaddaf9c3ced41ca4ea23a1025/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.1})

Using LangChain and HuggingFacePipeline for Prompting of text summarization

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              ```{text}```
              SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
!pip install datasets
from datasets import load_dataset

# Load the XSum dataset
dataset = load_dataset('xsum')

# Access the splits, e.g., 'test'
test_dataset = dataset['test']

test_df = test_dataset.to_pandas()

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
test_df.head()
# test_df.drop(columns=['id'], inplace=True)

,document,summary,id
0,"Prison Link Cymru had 1,099 referrals in 2015-...","There is a ""chronic"" need for more housing for...",38264402
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am...",34227252
2,"Jordan Hill, Brittany Covington and Tesfaye Co...",Four people accused of kidnapping and torturin...,38537698
3,The 48-year-old former Arsenal goalkeeper play...,West Brom have appointed Nicky Hammond as tech...,36175342
4,Restoring the function of the organ - which he...,The pancreas can be triggered to regenerate it...,39070183


In [ ]:
test_df.drop(columns=['id'], inplace=True)

In [ ]:
test_df.head()

,document,summary
0,"Prison Link Cymru had 1,099 referrals in 2015-...","There is a ""chronic"" need for more housing for..."
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am..."
2,"Jordan Hill, Brittany Covington and Tesfaye Co...",Four people accused of kidnapping and torturin...
3,The 48-year-old former Arsenal goalkeeper play...,West Brom have appointed Nicky Hammond as tech...
4,Restoring the function of the organ - which he...,The pancreas can be triggered to regenerate it...


In [ ]:
test_df['document'][0]

'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for acc

In [ ]:
test_df['summary'][0]

'There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.'

In [ ]:
# Create an empty column 'model_generated' in test_df to store the generated summaries
test_df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['document']
    summary = llm_chain.run(article_text)
    return summary

# Apply the generate_and_store_summary function to generate summaries for the first 25 articles
test_df.loc[:24, 'model_generated'] = test_df.loc[:24].apply(generate_and_store_summary, axis=1)

# Display the updated DataFrame with generated summaries for the first 25 articles
print(test_df[['document', 'model_generated']].head(25))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


                                             document  \
0   Prison Link Cymru had 1,099 referrals in 2015-...   
1   Officers searched properties in the Waterfront...   
2   Jordan Hill, Brittany Covington and Tesfaye Co...   
3   The 48-year-old former Arsenal goalkeeper play...   
4   Restoring the function of the organ - which he...   
5   But there certainly should be.\nThese are two ...   
6   Media playback is not supported on this device...   
7   It's no joke. But Kareem Badr says people did ...   
8   Relieved that the giant telecoms company would...   
9   "I'm really looking forward to it - the home o...   
10  The move is in response to an £8m cut in the s...   
11  The leaflets said the patient had been referre...   
12  Emily Thornberry said Labour would not "frustr...   
13  The National League sold the Republic of Irela...   
14  Iwan Wyn Lewis of Penygroes, Gwynedd, had been...   
15  The 33-year-old has featured only twice for th...   
16  Dr Waleed Abdalati told the

In [ ]:
test_df.head()

,document,summary,model_generated
0,"Prison Link Cymru had 1,099 referrals in 2015-...","There is a ""chronic"" need for more housing for...",The first paragraph is about\n -...
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am...",Police officers searched properties in Waterfr...
2,"Jordan Hill, Brittany Covington and Tesfaye Co...",Four people accused of kidnapping and torturin...,The four accused in the Chicago beating of an ...
3,The 48-year-old former Arsenal goalkeeper play...,West Brom have appointed Nicky Hammond as tech...,The former Arsenal goalkeeper Dave Richards pl...
4,Restoring the function of the organ - which he...,The pancreas can be triggered to regenerate it...,The organ which help with blood sugar levels c...


Randomly visualization of generated text

In [ ]:
test_df['model_generated'][11]

'A mix-up from an external printing company resulted in some patient appointment letters containing the wrong\n           leaflets. This information may have caused anxiety to those patients who were incorrectly informed that\n           "their symptoms may indicate\n'

In [ ]:
test_df['model_generated'][24]

'Two people got married at a Scottish ski resort before sliding off down a run in their wedding attire.\n'

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge-1': {'r': 0.2597657066404677, 'p': 0.12085234422570364, 'f': 0.1594068186594848}, 'rouge-2': {'r': 0.04075064032301884, 'p': 0.018740110611328357, 'f': 0.024237665565653276}, 'rouge-l': {'r': 0.2193738132046384, 'p': 0.10140823580703978, 'f': 0.13399362681501595}}


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)
print("BLEU Score for 25 Summaries:", bleu_score)


BLEU Score for 25 Summaries: 9.542118736121376e-232


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
!pip install bert_score

In [ ]:
from bert_score import score
# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)

# Print BERT Score
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.27 seconds, 7.65 sentences/sec
BERT Precision: 0.8282735347747803
BERT Recall: 0.8719192743301392
BERT F1 Score: 0.8489968776702881
